In [ ]:
!unzip /content/drive/MyDrive/AI_project_photo/archive.zip -d /content/drive/MyDrive/AI_project_photo/archieve

unzip:  cannot find or open /content/drive/MyDrive/AI_project_photo/archive.zip, /content/drive/MyDrive/AI_project_photo/archive.zip.zip or /content/drive/MyDrive/AI_project_photo/archive.zip.ZIP.


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import np_utils
from sklearn.utils import shuffle 
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings('ignore')

In [ ]:
def loaddataset(dataPath,category,photo_resize):
  img_name_list = os.listdir(dataPath)
  dataset=[(cv2.resize(cv2.imread(dataPath+img_name,cv2.IMREAD_GRAYSCALE),photo_resize,interpolation=cv2.INTER_AREA).reshape(-1,photo_resize[0],1),category) for img_name in img_name_list]#train oblong 724是gif，可以刪掉 test round desktop.ini
  return dataset
train_path='/content/drive/MyDrive/AI_project_photo/archieve/FaceShape Dataset/training_set'
test_path='/content/drive/MyDrive/AI_project_photo/archieve/FaceShape Dataset/testing_set'
photo_resize=(256,256)
# =======load train====================
train_data_0=loaddataset(train_path+'/Heart/',0,photo_resize)
train_data_1=loaddataset(train_path+'/Oblong/',1,photo_resize)
train_data_2=loaddataset(train_path+'/Oval/',2,photo_resize)
train_data_3=loaddataset(train_path+'/Round/',3,photo_resize)
train_data_4=loaddataset(train_path+'/Square/',4,photo_resize)
train_data=train_data_0+train_data_1+train_data_2+train_data_3+train_data_4
train_data_x=np.array([np.array(data[0]) for data in train_data])
train_data_y=[data[1] for data in train_data]
train_data_y=np_utils.to_categorical(train_data_y)
# =======load test====================
test_data_0=loaddataset(test_path+'/Heart/',0,photo_resize)
test_data_1=loaddataset(test_path+'/Oblong/',1,photo_resize)
test_data_2=loaddataset(test_path+'/Oval/',2,photo_resize)
test_data_3=loaddataset(test_path+'/Round/',3,photo_resize)
test_data_4=loaddataset(test_path+'/Square/',4,photo_resize)
test_data=test_data_0+test_data_1+test_data_2+test_data_3+test_data_4
test_data_x=np.array([np.array(data[0]) for data in test_data])
test_data_y=[data[1] for data in test_data]
test_data_y=np_utils.to_categorical(test_data_y)

train_data_x,train_data_y=shuffle(train_data_x,train_data_y,random_state=25)

FileNotFoundError: ignored

In [ ]:
model = Sequential()
model.add(Conv2D(filters=100, kernel_size=(2,2), strides=2, activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=100, kernel_size=(3,3), strides=2, activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=100, kernel_size=(4,4), strides=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(300,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 100)     500       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 100)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 100)       90100     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 100)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 100)         160100    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 100)        0

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
train_history=model.fit(x=train_data_x,y=train_data_y,validation_split=0.2,epochs=150,batch_size=200,verbose=2)

In [ ]:
scores = model.evaluate(test_data_x,test_data_y)
print('Test score:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
epochs=np.arange(1,151)
plt.figure(figsize=(8,4), dpi=80)
plt.subplot(1,2,1).set_title('Loss of each epoch')
plt.xlabel('epoch')
plt.plot(epochs,train_history.history['loss'])
plt.subplot(1,2,2).set_title('Accuracy of each epoch')
plt.xlabel('epoch')
plt.plot(epochs,train_history.history['accuracy'])
plt.show()

In [ ]:
def show_result(pred_y,real_y):
  heart=np.zeros(5)
  oblong=np.zeros(5)
  oval=np.zeros(5)
  round=np.zeros(5)
  square=np.zeros(5)
  for pre,real in zip(pred_y,real_y):
    if real==0:
      heart[pre]+=1
    elif real==1:
      oblong[pre]+=1
    elif real==2:
      oval[pre]+=1
    elif real==3:
      round[pre]+=1
    elif real==4:
      square[pre]+=1
  label=['heart','oblong','oval','round','square']
  plt.figure(figsize=(20,8), dpi=80)
  plt.subplot(1,5,1).set_title('Res of heart')
  plt.pie(heart,labels=label,autopct='%.2f%%') 
  plt.subplot(1,5,2).set_title('Res of oblong')
  plt.pie(oblong,labels=label,autopct='%.2f%%')
  plt.subplot(1,5,3).set_title('Res of oval')
  plt.pie(oval,labels=label, autopct='%.2f%%')
  plt.subplot(1,5,4).set_title('Res of round')
  plt.pie(round,labels=label, autopct='%.2f%%')
  plt.subplot(1,5,5).set_title('Res of square')
  plt.pie(square,labels=label,autopct='%.2f%%')
  plt.show()
prediction=model.predict(train_data_x)
pred_y=[np.argmax(p) for p in prediction]
real_y=[np.argmax(y) for y in train_data_y]
print('================================================================result of train===============================================================================')
show_result(pred_y,real_y)
prediction=model.predict(test_data_x)
pred_y=[np.argmax(p) for p in prediction]
real_y=[np.argmax(y) for y in test_data_y]
print('================================================================result of test =============================================================================')
show_result(pred_y,real_y)